In [ ]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
import pandas as pd

import tensorflow as tf
import keras
from keras import ops
from keras import layers
from tensorflow.keras import metrics

In [ ]:
# Path to the data directory
data_dir = "/content/handwriting-recognition/test_v2/test/"
train_csv = pd.read_csv('/content/handwriting-recognition//written_name_test_v2.csv')

train_csv = train_csv[train_csv['IDENTITY'] != 'UNREADABLE']
train_csv = train_csv[train_csv['IDENTITY'] != 'BLANK']
train_csv = train_csv.sample(frac=0.05, random_state=42)
print(train_csv.shape[0])

images = list(train_csv['FILENAME'])
labels = list(train_csv['IDENTITY'].astype(str))
train_csv = train_csv.dropna(subset=['IDENTITY'])

images = [os.path.join(data_dir, img) for img in images]

characters = set(char for label in labels if isinstance(label, str) for char in label)
characters = sorted(list(characters))

print("Number of images found: ", len(images))
print("Number of labels found: ", len(labels))
print("Number of unique characters: ", len(characters))
print("Characters present: ", characters)

# Batch size for training and validation
batch_size = 16

# Desired image dimensions
img_width = 200
img_height = 40

# Factor by which the image is going to be downsampled
# by the convolutional blocks. We will be using two
# convolution blocks and each block will have
# a pooling layer which downsample the features by a factor of 2.
# Hence total downsampling factor would be 4.
downsample_factor = 4
# Maximum length of any captcha in the dataset
max_length = max([len(str(label)) for label in train_csv['IDENTITY']])
print(max_length)
labels = [label.ljust(max_length) for label in labels]

In [ ]:
# Mapping characters to integers
char_to_num = layers.StringLookup(vocabulary=list(characters), mask_token=None)

# Mapping integers back to original characters
num_to_char = layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True
)

def split_data(images, labels, train_size=0.9, shuffle=True):
    # 1. Get the total size of the dataset
    size = len(images)
    # 2. Make an indices array and shuffle it, if required
    indices = ops.arange(size)
    if shuffle:
        keras.random.shuffle(indices)
    # 3. Get the size of training samples
    train_samples = int(size * train_size)
    # 4. Split data into training and validation sets
    x_train, y_train = images[indices[:train_samples]], labels[indices[:train_samples]]
    x_valid, y_valid = images[indices[train_samples:]], labels[indices[train_samples:]]
    x_test, y_test = images[indices[train_samples:]], labels[indices[train_samples:]]
    return x_train, x_valid, y_train, y_valid, x_test, y_test


# Splitting data into training and validation sets
x_train, x_valid, y_train, y_valid, x_test, y_test = split_data(np.array(images), np.array(labels))


def encode_single_sample(img_path, label):
    # 1. Read image
    img = tf.io.read_file(img_path)
    # 2. Decode and convert to grayscale
    img = tf.io.decode_png(img, channels=1)
    # 3. Convert to float32 in [0, 1] range
    img = tf.image.convert_image_dtype(img, tf.float32)
    # 4. Resize to the desired size
    img = tf.image.resize(img, [img_height, img_width], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    # 5. Transpose the image because we want the time
    # dimension to correspond to the width of the image.
    img = ops.transpose(img, axes=[1, 0, 2])
    # 6. Map the characters in label to numbers
    print(label)
    label = char_to_num(tf.strings.unicode_split(label, input_encoding="UTF-8"))
    # 7. Return a dict as our model is expecting two inputs
    print(label)
    return {"image": img, "label": label}

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = (
    train_dataset.map(encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

validation_dataset = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
validation_dataset = (
    validation_dataset.map(encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = (
    test_dataset.map(encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

In [ ]:
_, ax = plt.subplots(4, 4, figsize=(10, 5))
for batch in train_dataset.take(1):
    print('a')
    images = batch["image"]
    labels = batch["label"]
    for i in range(16):
        img = (images[i] * 255).numpy().astype("uint8")
        label = tf.strings.reduce_join(num_to_char(labels[i])).numpy().decode("utf-8")
        ax[i // 4, i % 4].imshow(img[:, :, 0].T, cmap="gray")
        ax[i // 4, i % 4].set_title(label)
        ax[i // 4, i % 4].axis("off")
plt.show()

In [ ]:
def ctc_batch_cost(y_true, y_pred, input_length, label_length):
    label_length = ops.cast(ops.squeeze(label_length, axis=-1), dtype="int32")
    input_length = ops.cast(ops.squeeze(input_length, axis=-1), dtype="int32")
    sparse_labels = ops.cast(
        ctc_label_dense_to_sparse(y_true, label_length), dtype="int32"
    )

    y_pred = ops.log(ops.transpose(y_pred, axes=[1, 0, 2]) + keras.backend.epsilon())

    return ops.expand_dims(
        tf.compat.v1.nn.ctc_loss(
            inputs=y_pred, labels=sparse_labels, sequence_length=input_length
        ),
        1,
    )


def ctc_label_dense_to_sparse(labels, label_lengths):
    label_shape = ops.shape(labels)
    num_batches_tns = ops.stack([label_shape[0]])
    max_num_labels_tns = ops.stack([label_shape[1]])

    def range_less_than(old_input, current_input):
        return ops.expand_dims(ops.arange(ops.shape(old_input)[1]), 0) < tf.fill(
            max_num_labels_tns, current_input
        )

    init = ops.cast(tf.fill([1, label_shape[1]], 0), dtype="bool")
    dense_mask = tf.compat.v1.scan(
        range_less_than, label_lengths, initializer=init, parallel_iterations=1
    )
    dense_mask = dense_mask[:, 0, :]

    label_array = ops.reshape(
        ops.tile(ops.arange(0, label_shape[1]), num_batches_tns), label_shape
    )
    label_ind = tf.compat.v1.boolean_mask(label_array, dense_mask)

    batch_array = ops.transpose(
        ops.reshape(
            ops.tile(ops.arange(0, label_shape[0]), max_num_labels_tns),
            tf.reverse(label_shape, [0]),
        )
    )
    batch_ind = tf.compat.v1.boolean_mask(batch_array, dense_mask)
    indices = ops.transpose(
        ops.reshape(ops.concatenate([batch_ind, label_ind], axis=0), [2, -1])
    )

    vals_sparse = tf.compat.v1.gather_nd(labels, indices)

    return tf.SparseTensor(
        ops.cast(indices, dtype="int64"),
        vals_sparse,
        ops.cast(label_shape, dtype="int64")
    )

@keras.saving.register_keras_serializable(package="customs", name="CTCLayer")
class CTCLayer(layers.Layer):
    # def __init__(self, name=None):
    #     super().__init__(name=name)
    #     self.loss_fn = ctc_batch_cost
    def __init__(self, trainable=True, dtype='float32', name=None, **kwargs):
        super().__init__(trainable=trainable, dtype=dtype, name=name, **kwargs)
        self.loss_fn = ctc_batch_cost

    def call(self, y_true, y_pred):
        # Compute the training-time loss value and add it
        # to the layer using `self.add_loss()`.
        batch_len = ops.cast(ops.shape(y_true)[0], dtype="int64")
        input_length = ops.cast(ops.shape(y_pred)[1], dtype="int64")
        label_length = ops.cast(ops.shape(y_true)[1], dtype="int64")

        input_length = input_length * ops.ones(shape=(batch_len, 1), dtype="int64")
        label_length = label_length * ops.ones(shape=(batch_len, 1), dtype="int64")

        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
        self.add_loss(loss)

        # At test time, just return the computed predictions
        return y_pred

def build_model(num_conv_layers=1, filter_size=3, activation='relu'):
    # Inputs to the model
    input_img = layers.Input(
        shape=(img_width, img_height, 1), name="image", dtype="float32"
    )
    labels = layers.Input(name="label", shape=(None,), dtype="float32")

    # First conv block
    x = layers.Conv2D(
        64,
        (filter_size, filter_size),
        activation=activation,
        kernel_initializer="he_normal",
        padding="same",
        name="Conv1",
    )(input_img)
    x = layers.MaxPooling2D((2, 2), name="pool1")(x)

    # Usuń drugą warstwę konwolucyjną i odpowiadający jej blok pooleowania
    # Druga warstwa konwolucyjna
    # x = layers.Conv2D(
    #     64,
    #     (filter_size, filter_size),
    #     activation=activation,
    #     kernel_initializer="he_normal",
    #     padding="same",
    #     name="Conv2",
    # )(x)
    # x = layers.MaxPooling2D((2, 2), name="pool2")(x)

    # We have used one max pool with pool size and strides 2.
    # Hence, downsampled feature maps are 2x smaller. The number of
    # filters in the last layer is 64. Reshape accordingly before
    # passing the output to the RNN part of the model
    new_shape = ((img_width // 2), (img_height // 2) * 64)  # Zmiana rozmiaru tensora po usunięciu drugiej warstwy konwolucyjnej
    x = layers.Reshape(target_shape=new_shape, name="reshape")(x)
    x = layers.Dense(64, activation="relu", name="dense1")(x)
    x = layers.Dropout(0.2)(x)

    # RNNs
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True, dropout=0.25))(x)
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True, dropout=0.25))(x)

    # Output layer
    x = layers.Dense(
        len(char_to_num.get_vocabulary()) + 1, activation="softmax", name="dense2"
    )(x)

    # Add CTC layer for calculating CTC loss at each step
    output = CTCLayer(name="ctc_loss")(labels, x)

    # Define the model
    model = keras.models.Model(
        inputs=[input_img, labels], outputs=output, name="ocr_model_v1"
    )
    # Optimizer
    opt = keras.optimizers.Adam()
    # Compile the model and return
    model.compile(optimizer=opt)
    return model


# Get the model
model = build_model()
model.summary()


In [ ]:
def character_error_rate(y_true, y_pred):
    # Convert integer labels back to strings
    y_true = tf.strings.reduce_join(num_to_char(y_true), axis=-1)
    y_pred = tf.strings.reduce_join(num_to_char(y_pred), axis=-1)

    # Compute Levenshtein distance between true and predicted strings
    distances = tf.edit_distance(y_true, y_pred, normalize=False)

    # Compute total number of characters in true labels
    total_characters = tf.strings.length(y_true)

    # Calculate CER as the average distance normalized by total number of characters
    cer = tf.reduce_mean(distances / total_characters)

    return cer

def word_error_rate(y_true, y_pred):
    # Convert integer labels back to strings
    y_true = tf.strings.reduce_join(num_to_char(y_true), axis=-1)
    y_pred = tf.strings.reduce_join(num_to_char(y_pred), axis=-1)

    # Split true and predicted strings into words
    true_words = tf.strings.split(y_true)
    pred_words = tf.strings.split(y_pred)

    # Compute Levenshtein distance between true and predicted words
    distances = tf.edit_distance(true_words, pred_words, normalize=False)

    # Compute total number of words in true labels
    total_words = tf.cast(tf.size(true_words), tf.float32)

    # Calculate WER as the average distance normalized by total number of words
    wer = tf.reduce_mean(distances / total_words)

    return wer

def token_error_rate(y_true, y_pred):
    # Convert integer labels back to strings
    y_true = tf.strings.reduce_join(num_to_char(y_true), axis=-1)
    y_pred = tf.strings.reduce_join(num_to_char(y_pred), axis=-1)

    # Tokenize true and predicted strings
    true_tokens = tf.strings.regex_replace(y_true, r"([.,!?;])", r" \1 ")  # Add space around punctuation
    true_tokens = tf.strings.split(true_tokens)
    pred_tokens = tf.strings.regex_replace(y_pred, r"([.,!?;])", r" \1 ")  # Add space around punctuation
    pred_tokens = tf.strings.split(pred_tokens)

    # Compute Levenshtein distance between true and predicted tokens
    distances = tf.edit_distance(true_tokens, pred_tokens, normalize=False)

    # Compute total number of tokens in true labels
    total_tokens = tf.cast(tf.size(true_tokens), tf.float32)

    # Calculate TER as the average distance normalized by total number of tokens
    ter = tf.reduce_mean(distances / total_tokens)

    return ter


In [ ]:
# TODO restore epoch count.
epochs = 5
early_stopping_patience = 10
# Add early stopping
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=early_stopping_patience, restore_best_weights=True
)

# Train the model
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=epochs,
    callbacks=[early_stopping],
)

Epoch 1/5
117/117 ━━━━━━━━━━━━━━━━━━━━ 66s 476ms/step - loss: 727.7364 - val_loss: 418.9305
Epoch 2/5
117/117 ━━━━━━━━━━━━━━━━━━━━ 82s 476ms/step - loss: 402.5636 - val_loss: 409.3893
Epoch 3/5
 57/117 ━━━━━━━━━━━━━━━━━━━━ 24s 416ms/step - loss: 395.4935

In [ ]:
model.save("trained_model.keras")

In [ ]:
predictions = model.predict(test_dataset)

# def decode_predictions(predictions):
#     decoded_predictions = []
#     for prediction in predictions:
#         # Numery do znaków
#         decoded_labels = num_to_char(prediction)
#         # Połączenie znaków w string
#         decoded_string = tf.strings.reduce_join(decoded_labels, axis=-1)
#         # Dodanie zdekodowanej etykiety do listy
#         decoded_predictions.append(decoded_string)
#     return decoded_predictions

# # Dekodowanie przewidywań
# decoded_predictions = decode_predictions(predictions)
# print(decoded_predictions)

# Mapping characters to integers

In [ ]:
def ctc_decode(y_pred, input_length, greedy=True, beam_width=100, top_paths=1):
    input_shape = ops.shape(y_pred)
    num_samples, num_steps = input_shape[0], input_shape[1]
    y_pred = ops.log(ops.transpose(y_pred, axes=[1, 0, 2]) + keras.backend.epsilon())
    input_length = ops.cast(input_length, dtype="int32")

    if greedy:
        (decoded, log_prob) = tf.nn.ctc_greedy_decoder(
            inputs=y_pred, sequence_length=input_length
        )
    else:
        (decoded, log_prob) = tf.compat.v1.nn.ctc_beam_search_decoder(
            inputs=y_pred,
            sequence_length=input_length,
            beam_width=beam_width,
            top_paths=top_paths,
        )
    decoded_dense = []
    for st in decoded:
        st = tf.SparseTensor(st.indices, st.values, (num_samples, num_steps))
        decoded_dense.append(tf.sparse.to_dense(sp_input=st, default_value=-1))
    return (decoded_dense, log_prob)


# Get the prediction model by extracting layers till the output layer
prediction_model = keras.models.Model(
    model.input[0], model.get_layer(name="dense2").output
)
prediction_model.summary()


# A utility function to decode the output of the network
def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    # Use greedy search. For complex tasks, you can use beam search
    results = ctc_decode(pred, input_length=input_len, greedy=True)[0][0][
        :, :max_length
    ]
    # Iterate over the results and get back the text
    output_text = []
    for res in results:
        res = tf.strings.reduce_join(num_to_char(res)).numpy().decode("utf-8")
        output_text.append(res)
    return output_text


#  Let's check results on some validation samples
for batch in validation_dataset.take(1):
    batch_images = batch["image"]
    batch_labels = batch["label"]

    preds = prediction_model.predict(batch_images)
    pred_texts = decode_batch_predictions(preds)

    orig_texts = []
    for label in batch_labels:
        label = tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
        orig_texts.append(label)

    _, ax = plt.subplots(4, 4, figsize=(15, 5))
    for i in range(len(pred_texts)):
        img = (batch_images[i, :, :, 0] * 255).numpy().astype(np.uint8)
        img = img.T
        title = f"Prediction: {pred_texts[i]}\nReal: {orig_texts[i]}"
        ax[i // 4, i % 4].imshow(img, cmap="gray")
        ax[i // 4, i % 4].set_title(title)
        ax[i // 4, i % 4].axis("off")
plt.show()

In [ ]:
def levenshtein(s1, s2):
    dp = [[0] * (len(s2) + 1) for _ in range(len(s1) + 1)]

    for i in range(len(s1) + 1):
        dp[i][0] = i
    for j in range(len(s2) + 1):
        dp[0][j] = j

    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            if s1[i - 1] == s2[j - 1]:
                cost = 0
            else:
                cost = 1
            dp[i][j] = min(dp[i - 1][j] + 1,
                           dp[i][j - 1] + 1,
                           dp[i - 1][j - 1] + cost)
    return dp[len(s1)][len(s2)]

In [ ]:
total_distance = 0
num_samples = 0
for batch in validation_dataset:
    batch_images = batch["image"]
    batch_labels = batch["label"]

    preds = prediction_model.predict(batch_images)
    pred_texts = decode_batch_predictions(preds)

    orig_texts = []
    for label in batch_labels:
        label = tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
        orig_texts.append(label)

    for i in range(len(pred_texts)):
        total_distance += levenshtein(pred_texts[i], orig_texts[i])
        num_samples += 1


# Calculate and print the average Levenshtein distance
average_distance = total_distance / num_samples
print(f"Average Levenshtein Distance: {average_distance}")